project

Open-domain systems


Setup


In [ ]:
!pip install torch
!pip install datasets transformers
!pip install sklearn

     |████████████████████████████████| 306 kB 27.4 MB/s 
     |████████████████████████████████| 3.4 MB 46.3 MB/s 
     |████████████████████████████████| 133 kB 34.9 MB/s 
     |████████████████████████████████| 1.1 MB 58.8 MB/s 
     |████████████████████████████████| 243 kB 62.9 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 58.0 MB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
     |████████████████████████████████| 3.3 MB 63.1 MB/s 
     |████████████████████████████████| 192 kB 81.2 MB/s 
     |████████████████████████████████| 271 kB 73.2 MB/s 
     |████████████████████████████████| 160 kB 57.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizerFast
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

In [ ]:
checkpoint = "distilbert-base-uncased"

Data Preparation


In [ ]:
dataset = load_dataset("boolq")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset boolq downloaded and prepared to /root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
DatasetDict({
    train:Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427})
    validation:Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270})
})

SyntaxError: ignored

In [ ]:
{''''answer': True,'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), 
is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. 
It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan 
(officially known as Tajiki since the Soviet era), and some other regions which historically 
were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, 
a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.',
'question': 'do iran and afghanistan speak the same language'''}

{"'answer': True,'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), \nis one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. \nIt is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan \n(officially known as Tajiki since the Soviet era), and some other regions which historically \nwere Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, \na modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.',\n'question': 'do iran and afghanistan speak the same language"}

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
  encoded = tokenizer(example["question"], example["passage"], truncation=True)
  encoded["labels"] = [int(a) for a in example["answer"]]
  return encoded

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Model definition and training


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [ ]:
args = TrainingArguments("roberta-booql",
                         per_device_train_batch_size=16,
                         learning_rate=1e-3,
                         num_train_epochs=3)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"],
                  data_collator=data_collator,
                  tokenizer=tokenizer,)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: answer, passage, question.
***** Running training *****
  Num examples = 9427
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1770


Step,Training Loss


Evaluation


In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
y_pred = predictions.predictions.argmax(-1)
labels = predictions.label_ids

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: passage, question, answer.
***** Running Prediction *****
  Num examples = 3270
  Batch size = 8


In [ ]:
metric = load_metric("accuracy")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
metric.compute(predictions=y_pred, references=predictions.label_ids)

{'accuracy': 0.6207951070336392}

In [ ]:
{'accuracy': 0.7327217125382263}

{'accuracy': 0.7327217125382263}